In [7]:
# Importing the necessary libraries and packages
# Importing the VGG16 model from Keras
from keras.applications.vgg16 import VGG16 

# Importing the ResNet50 model from TensorFlow
from tensorflow.keras.applications.resnet50 import ResNet50 

# Importing the Model class from Keras
from keras.models import Model                 

# Importing the image preprocessing module from Keras
from keras.preprocessing import image          

# Importing the necessary layers from TensorFlow Keras
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout 

# Importing NumPy for numerical operations
import numpy as np                             

# Importing TensorFlow
import tensorflow as tf                        

# Importing Matplotlib for visualization
import matplotlib.pyplot as plt                

# Configuring Matplotlib for inline plotting
%matplotlib inline                              

# Importing the os module for file operations
import os                                      

 # Importing OpenCV for image processing
import cv2                                    

# Setting the directory path for training data
train_dir = "./training_set"

# Setting the directory path for test data
eval_dir = "./test_set"


In [ ]:
#Helper function to load images from given directories
import keras
def load_images(directory):
    images = []
    labels = []
    for idx, label in enumerate(uniq_labels):
        
        for file in os.listdir(directory + "/" + label):
            filepath = directory + "/" + label + "/" + file
            image = cv2.resize(cv2.imread(filepath), (64, 64))
            images.append(image)
            labels.append(idx)
    images = np.array(images)
    labels = np.array(labels)
    return(images, labels)

uniq_labels = sorted(os.listdir(train_dir))
uniq_labels.remove('.DS_Store')

images, labels = load_images(directory = train_dir)

uniq_labels_eval=sorted(os.listdir(eval_dir))
uniq_labels_eval.remove('.DS_Store')

if uniq_labels == uniq_labels_eval :
    X_eval, y_eval = load_images(directory = eval_dir)

In [ ]:
uniq_labels

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size = 0.2, stratify = labels)

n = len(uniq_labels)
train_n = len(X_train)
test_n = len(X_test)

print("Total number of symbols: ", n)
print("Number of training images: " , train_n)
print("Number of testing images: ", test_n)

eval_n = len(X_eval)
print("Number of evaluation images: ", eval_n)

In [ ]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)
y_eval = keras.utils.to_categorical(y_eval)

In [ ]:
print(y_train[0])
print(len(y_train[0]))

In [ ]:
X_train = X_train.astype('float32')/255.0
X_test = X_test.astype('float32')/255.0
X_eval = X_eval.astype('float32')/255.0

In [ ]:
#Initialising vgg16 
classifier_vgg16 = VGG16(input_shape= (64,64,3),include_top=False,weights='imagenet')

In [ ]:
#Initialising resnet50 
classifier_resnet = ResNet50(input_shape=(64,64,3),include_top=False,weights='imagenet')

In [ ]:
#don't train existing weights for vgg16
for layer in classifier_vgg16.layers:
    layer.trainable = False

#don't train existing weights for resnet50
for layer in classifier_resnet.layers:
    layer.trainable = False

In [ ]:
#VGG16
classifier1 = classifier_vgg16.output#head mode
classifier1 = Flatten()(classifier1)#adding layer of flatten
classifier1 = Dense(units=256, activation='relu')(classifier1)
classifier1 = Dropout(0.6)(classifier1)
classifier1 = Dense(units=39, activation='softmax')(classifier1)

model = Model(inputs = classifier_vgg16.input , outputs = classifier1)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#summary of vgg16
model.summary()


In [ ]:
#fit the model
#it will take some time to train
#vgg16
history = model.fit(X_train, y_train, epochs =1, batch_size =5,validation_data=(X_test,y_test))

In [ ]:
# Saving the model of vgg16
model.save('model_vgg16.h5')

In [ ]:
#resnet50
classifier2 = classifier_resnet.output#head mode
classifier2 = Flatten()(classifier2)#adding layer of flatten
classifier2 = Dropout(0.6)(classifier2)
classifier2 = Dense(units=39, activation='softmax')(classifier2)

model2 = Model(inputs = classifier_resnet.input , outputs = classifier2)
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#summary of resnet50
model2.summary()

In [ ]:
#fit the model
#resnet50
history2 = model2.fit(X_train, y_train, epochs =1, batch_size = 5,validation_data=(X_test,y_test))

In [ ]:

# Saving the model of resnet
model2.save('model_resnet.h5')

In [ ]:
score = model.evaluate(x = X_test, y = y_test, verbose = 0)
print('Accuracy for test images:', round(score[1]*100, 3), '%')
score = model.evaluate(x = X_eval, y = y_eval, verbose = 0)
print('Accuracy for evaluation images:', round(score[1]*100, 3), '%')

score = model2.evaluate(x = X_test, y = y_test, verbose = 0)
print('Accuracy for test images:', round(score[1]*100, 3), '%')
score = model2.evaluate(x = X_eval, y = y_eval, verbose = 0)
print('Accuracy for evaluation images:', round(score[1]*100, 3), '%')

In [ ]:
#vgg16
# summarize history for accuracy
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy of vgg16')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
#resnet50
# summarize history for accuracy
import matplotlib.pyplot as plt
plt.plot(history2.history['accuracy'])
plt.plot(history2.history['val_accuracy'])
plt.title('model accuracy of resnet50')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
import gradio as gr
from tensorflow.keras.models import load_model

# Load the saved model
model_VGG = load_model('model_vgg16.h5')

In [ ]:
import numpy as np
import keras.utils as image

#class_names=['1','10','2','3','4','5','6','7','8','9','A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','Best of luck','I love you','space']




In [ ]:

def predict_image(img):
  res=[]
  #print(img.shape)
  img_4d=img.reshape(-1,64,64,3)
  result = model_VGG.predict(img_4d)

  for i,j in enumerate(range(result.shape[1])):
        #print(result[0][i],j,)
        
    if result[0][i]==1.0:
      res=uniq_labels[j]
      print(res)
  return res


In [6]:
image = gr.inputs.Image(shape=(64,64))
label = gr.outputs.Label(num_top_classes=5)

gr.Interface(fn=predict_image, inputs=image, outputs=label,interpretation='default').launch(debug='True')

Keyboard interruption in main thread... closing server.
